In [16]:
import pandas as pd
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Bank Telemarketing
Determine who will subscribe a term deposit.

The dataset for this competition is from the UCI Machine Learning Repository.  The original source is cited in the data dictionary.  Please do not use the original data during this competition, but rather use the training data provided here.  The data have been modified so that patterns in the original data may not hold.

Use the training data to build your models.  Apply your models to the test data and submit a prediction (the prediction column can have values a) 0 = "no" and 1 = "yes",  b) a probability of a yes, or c) a ranking where a higher number means yes).

Each group will present their results during the first class session on April 21st. Requirements regarding the presentation are on Blackboard.


Input variables:
#### bank client data:
1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
7. loan: has personal loan? (categorical: 'no','yes','unknown')

#### related with the last contact of the current campaign:
8. contact: contact communication type (categorical: 'cellular','telephone') 
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#### other attributes:
12. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### social and economic context attributes
16. emp.var.rate: employment variation rate - quarterly indicator (numeric)
17. cons.price.idx: consumer price index - monthly indicator (numeric) 
18. cons.conf.idx: consumer confidence index - monthly indicator (numeric) 
19. euribor3m: euribor 3 month rate - daily indicator (numeric)
20. nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):

y - has the client subscribed a term deposit? (binary: 'yes','no')

In [12]:
df = pd.read_excel('bankTestReport.xlsx', sheetname='Train')
df['y'] = df.apply(lambda row: 0 if str(row.y) == 'no'else 1, axis=1)
df_train, df_test =  train_test_split(df, test_size=0.33, random_state=42)
df_train.head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1767,32,management,married,university.degree,no,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,0
2050,37,self-employed,married,basic.9y,no,yes,no,cellular,aug,fri,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,0
57,31,admin.,married,university.degree,no,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1,0
1621,34,admin.,married,university.degree,no,no,no,telephone,may,tue,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.856,5191.0,0
2420,36,technician,single,professional.course,no,no,no,cellular,jul,thu,...,4,999,0,nonexistent,1.4,93.918,-42.7,4.958,5228.1,0


### Факторы: значения

In [13]:
for column in df_train.columns:
    factor_values = sorted(df_train[column].unique().tolist())
    if len(factor_values) < 20:
        print(f"{column}: {factor_values}")
    else:
        print(f"{column}: [{min(factor_values)}: {max(factor_values)}]")

age: [18: 88]
job: ['admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown']
marital: ['divorced', 'married', 'single', 'unknown']
education: ['basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'professional.course', 'university.degree', 'unknown']
default: ['no', 'unknown']
housing: ['no', 'unknown', 'yes']
loan: ['no', 'unknown', 'yes']
contact: ['cellular', 'telephone']
month: ['apr', 'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep']
day_of_week: ['fri', 'mon', 'thu', 'tue', 'wed']
duration: [5: 3643]
campaign: [1: 24]
pdays: [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 16, 999]
previous: [0, 1, 2, 3, 4, 5, 6]
poutcome: ['failure', 'nonexistent', 'success']
emp.var.rate: [-3.4, -3.0, -2.9, -1.8, -1.7, -1.1, -0.2, -0.1, 1.1, 1.4]
cons.price.idx: [92.201: 94.767]
cons.conf.idx: [-50.8: -26.9]
euribor3m: [0.635: 4.97]
nr.employed: [4963.6, 4991.6, 5008.7, 5017.5, 5023.5, 5076.2,

### Выделение количественнных и категориальных факторов

In [14]:
quality_factors = ['age',
                   'duration',
                   'campaign',
                   'pdays',
                   'previous',
                   'emp.var.rate',
                   'cons.price.idx',
                   'cons.conf.idx',
                   'euribor3m',
                   'nr.employed'
                  ]
categorial_factors = list(set(df_train.columns) - set(quality_factors) - set('y')) 

### Простейшая модель: линенйая регрессия на основе количественнных факторов
Посмотрим качество модели на основе существующих количественных факторов, без дполнительной работы по преобразованию данных, чтобы было с чем сравнивать в дальнейшем. Метрика оценки качество модели - площадь под ROC-кривой, которая показывает зависимость количества верно классифицированных положительных примеров от количества неверно классифицированных отрицательных примеров. 

In [15]:
model = SGDClassifier().fit(df_train[quality_factors], df_train['y'])
print('roc auc train')
print(roc_auc_score(df_train['y'], model.predict(df_train[quality_factors])))
print('roc auc test')
print(roc_auc_score(df_test['y'], model.predict(df_test[quality_factors])))

roc auc train
0.7595628595338662
roc auc test
0.732038338658147


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
